In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding,Dense ,Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

 a. Data preparation

In [2]:
with open("./CBOW.txt","r") as file:
    text=file.read()

Preprocessing text

In [3]:
sentences=text.split('.')
sentences=[s.lower() for s in sentences]
sentences

['the speed of transmission is an important point of difference between the two viruses',
 ' influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than covid-19 virus',
 ' the serial interval for covid-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days',
 ' this means that influenza can spread faster than covid-19',
 ' \n\nfurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza',
 ' in contrast, while we are learning that there are people who can shed covid-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission',
 ' \n\nthe reproductive number â€“ the number of secondary infections generated from one infected individ

In [4]:
# Tokenizing the sentences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences=tokenizer.texts_to_sequences(sentences)
print(sequences)

[[1, 38, 2, 8, 9, 39, 40, 41, 2, 42, 13, 1, 43, 23], [3, 44, 11, 24, 45, 46, 47, 1, 14, 25, 48, 10, 26, 2, 27, 12, 11, 24, 15, 16, 1, 14, 13, 49, 50, 17, 4, 5, 6], [1, 15, 16, 7, 4, 5, 6, 9, 51, 10, 18, 19, 52, 20, 28, 7, 3, 6, 1, 15, 16, 9, 29, 20], [30, 53, 31, 3, 32, 54, 55, 17, 4, 5], [56, 8, 33, 1, 57, 29, 19, 20, 2, 58, 59, 60, 61, 62, 8, 63, 2, 1, 6, 64, 1, 26, 2, 27, 21, 9, 11, 34, 35, 2, 8, 7, 3], [33, 65, 28, 66, 22, 67, 31, 68, 22, 69, 70, 32, 71, 4, 5, 6, 72, 73, 74, 75, 10, 76, 77, 78, 79, 30, 80, 81, 82, 10, 18, 11, 34, 35, 2, 8], [1, 83, 36, 21, 1, 36, 2, 84, 85, 86, 25, 87, 88, 89, 21, 9, 90, 10, 18, 13, 37, 12, 37], [19, 7, 4, 5, 6, 91, 17, 7, 3], [92, 93, 7, 94, 4, 5, 12, 3, 23, 22, 95, 96, 12, 14, 97, 98, 99, 100, 101, 102], []]


In [5]:
index_to_word=tokenizer.index_word
word_to_index=tokenizer.word_index
print(index_to_word,"\n")
print(word_to_index)

{1: 'the', 2: 'of', 3: 'influenza', 4: 'covid', 5: '19', 6: 'virus', 7: 'for', 8: 'transmission', 9: 'is', 10: 'to', 11: 'a', 12: 'and', 13: 'between', 14: 'time', 15: 'serial', 16: 'interval', 17: 'than', 18: 'be', 19: '5', 20: 'days', 21: 'â€“', 22: 'are', 23: 'viruses', 24: 'shorter', 25: 'from', 26: 'appearance', 27: 'symptoms', 28: 'while', 29: '3', 30: 'this', 31: 'that', 32: 'can', 33: 'in', 34: 'major', 35: 'driver', 36: 'number', 37: '2', 38: 'speed', 39: 'an', 40: 'important', 41: 'point', 42: 'difference', 43: 'two', 44: 'has', 45: 'median', 46: 'incubation', 47: 'period', 48: 'infection', 49: 'successive', 50: 'cases', 51: 'estimated', 52: '6', 53: 'means', 54: 'spread', 55: 'faster', 56: 'further', 57: 'first', 58: 'illness', 59: 'or', 60: 'potentially', 61: 'pre', 62: 'symptomatic', 63: 'â€“transmission', 64: 'before', 65: 'contrast', 66: 'we', 67: 'learning', 68: 'there', 69: 'people', 70: 'who', 71: 'shed', 72: '24', 73: '48', 74: 'hours', 75: 'prior', 76: 'symptom', 77

b. Generate training data ||
Creating contexts and targets

In [6]:
vocab_size=len(word_to_index)+1
emb_size=100
context_size=2
contexts=[]
targets=[]
for sequence in sequences:
  for i in range(context_size,len(sequence)-context_size):
    context=[sequence[i-2],sequence[i-1],sequence[i+1],sequence[i+2]]
    target=sequence[i]
    contexts.append(context)
    targets.append(target)

print(contexts,"\n")
print(targets)


[[1, 38, 8, 9], [38, 2, 9, 39], [2, 8, 39, 40], [8, 9, 40, 41], [9, 39, 41, 2], [39, 40, 2, 42], [40, 41, 42, 13], [41, 2, 13, 1], [2, 42, 1, 43], [42, 13, 43, 23], [3, 44, 24, 45], [44, 11, 45, 46], [11, 24, 46, 47], [24, 45, 47, 1], [45, 46, 1, 14], [46, 47, 14, 25], [47, 1, 25, 48], [1, 14, 48, 10], [14, 25, 10, 26], [25, 48, 26, 2], [48, 10, 2, 27], [10, 26, 27, 12], [26, 2, 12, 11], [2, 27, 11, 24], [27, 12, 24, 15], [12, 11, 15, 16], [11, 24, 16, 1], [24, 15, 1, 14], [15, 16, 14, 13], [16, 1, 13, 49], [1, 14, 49, 50], [14, 13, 50, 17], [13, 49, 17, 4], [49, 50, 4, 5], [50, 17, 5, 6], [1, 15, 7, 4], [15, 16, 4, 5], [16, 7, 5, 6], [7, 4, 6, 9], [4, 5, 9, 51], [5, 6, 51, 10], [6, 9, 10, 18], [9, 51, 18, 19], [51, 10, 19, 52], [10, 18, 52, 20], [18, 19, 20, 28], [19, 52, 28, 7], [52, 20, 7, 3], [20, 28, 3, 6], [28, 7, 6, 1], [7, 3, 1, 15], [3, 6, 15, 16], [6, 1, 16, 9], [1, 15, 9, 29], [15, 16, 29, 20], [30, 53, 3, 32], [53, 31, 32, 54], [31, 3, 54, 55], [3, 32, 55, 17], [32, 54, 17,

In [7]:
# Printing features with target
for i in range(5):
  words=[]
  target=index_to_word.get(targets[i])
  for j in contexts[i]:
    words.append(index_to_word.get(j))
  print(words,"=>",target)

['the', 'speed', 'transmission', 'is'] => of
['speed', 'of', 'is', 'an'] => transmission
['of', 'transmission', 'an', 'important'] => is
['transmission', 'is', 'important', 'point'] => an
['is', 'an', 'point', 'of'] => important


Creating training and testing data


In [8]:
x=np.array(contexts)
y=np.array(targets)
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (162, 4)
Shape of y: (162,)


c. Train model  || *Defining model*

In [9]:
model=Sequential([
    Embedding(input_dim=vocab_size,output_dim=emb_size,input_length=context_size*2),
    Lambda(lambda x:tf.reduce_mean(x,axis=1)),
    Dense(256,activation='relu'),
    Dense(512,activation='relu'),
    Dense(vocab_size,activation='softmax')
])

c:\Users\Atharva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
history=model.fit(x,y,epochs=20)

Epoch 1/20

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0290 - loss: 4.6330
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1446 - loss: 4.6070 
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1278 - loss: 4.5725 
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1072 - loss: 4.5156 
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1260 - loss: 4.3900 
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0496 - loss: 4.2648     
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0552 - loss: 4.1055     
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0810 - loss: 4.0387 
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0838 - loss: 3.9049 
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1639 - loss: 3.7388 
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2062 - loss: 3.6011 
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1795 - loss: 

 d. Output

In [12]:
test_words=[index_to_word[index] for index in contexts[0]]

input_data=np.expand_dims(contexts[0],axis=0)
print(input_data)

pred=model.predict(input_data)
predicted_index=np.argmax(pred[0])

print("Context words:", test_words)
print("Predicted terget word : ",index_to_word[predicted_index])


[[ 1 38  8  9]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Context words: ['the', 'speed', 'transmission', 'is']
Predicted terget word :  of
